# Building a Google Sheets Agent with OpenAI Agents SDK

In this tutorial, we'll build a powerful spreadsheet agent that can interact with Google Sheets to read, write, and manipulate data. This agent will use the OpenAI Agents SDK with custom tools for Google Sheets integration.

## What We'll Learn

1. **Google Sheets Setup** - Configure Google Sheets API credentials
2. **Custom Sheets Tools** - Create function tools for Google Sheets operations
3. **Spreadsheet Agent** - Build an intelligent agent for data analysis and manipulation
4. **Practical Examples** - Real-world use cases and workflows

## Prerequisites

- Python 3.11+
- OpenAI API key
- Google Cloud Platform account
- Google Sheets API enabled
- Service account credentials (credentials.json)

## Google Sheets API Setup

**Important: Before running this notebook, you need to:**

1. **Create a Google Cloud Project**:
   - Go to [Google Cloud Console](https://console.cloud.google.com/)
   - Create a new project or select an existing one

2. **Enable Google Sheets API**:
   - Navigate to "APIs & Services" > "Library"
   - Search for "Google Sheets API" and enable it

3. **Create Service Account Credentials**:
   - Go to "APIs & Services" > "Credentials"
   - Click "Create Credentials" > "Service Account"
   - Fill in the details and create the service account
   - Click on the service account, go to "Keys" tab
   - Click "Add Key" > "Create new key" > "JSON"
   - Save the downloaded file as `credentials.json` in your project directory

4. **Share Your Spreadsheet**:
   - Open your Google Sheet
   - Share it with the service account email (found in credentials.json)
   - Give "Editor" permissions

## 1. Installation and Setup

First, let's install the required packages and set up our environment.

In [3]:
# Install required packages
# !pip install openai-agents google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2

# Import required libraries
import os
import json
from typing import List, Dict, Any, Optional
from datetime import datetime
import pandas as pd

# OpenAI Agents SDK
from agents import Agent, Runner, function_tool, trace
from agents import ModelSettings
from agents.run import RunConfig

# Google Sheets API
import googleapiclient.discovery
from google.oauth2 import service_account
from googleapiclient.errors import HttpError

# Set your OpenAI API key
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

print("All packages imported successfully!")

All packages imported successfully!


## 2. Google Sheets Service Setup

Let's set up the Google Sheets API connection using service account credentials.

In [ ]:
# Google Sheets setup
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
CREDENTIALS_FILE = '../credentials.json'  # Path to your service account credentials

def setup_sheets_service():
    """Initialize Google Sheets service with service account credentials."""
    try:
        credentials = service_account.Credentials.from_service_account_file(
            CREDENTIALS_FILE, scopes=SCOPES
        )
        service = googleapiclient.discovery.build('sheets', 'v4', credentials=credentials)
        return service
    except FileNotFoundError:
        print("❌ Error: credentials.json not found!")
        print("Please download your service account credentials from Google Cloud Console")
        print("and save them as 'credentials.json' in this directory.")
        return None
    except Exception as e:
        print(f"❌ Error setting up Google Sheets service: {e}")
        return None

# Initialize the service
sheets_service = setup_sheets_service()

if sheets_service:
    print("✅ Google Sheets service initialized successfully!")
else:
    print("⚠️ Google Sheets service not available. Please check your credentials.")

❌ Error setting up Google Sheets service: Service account info was not in the expected format, missing fields token_uri, client_email.
⚠️ Google Sheets service not available. Please check your credentials.


## 3. Custom Google Sheets Tools

Now let's create custom tools for Google Sheets operations using the `@function_tool` decorator.

In [ ]:
@function_tool
def read_sheet_data(spreadsheet_id: str, range_name: str = "A:Z") -> str:
    """
    Read data from a Google Sheet.
    
    Args:
        spreadsheet_id: The ID of the Google Spreadsheet (from the URL)
        range_name: The range to read (e.g., 'Sheet1!A1:C10' or 'A:Z')
    
    Returns:
        String representation of the sheet data
    """
    if not sheets_service:
        return "Error: Google Sheets service not available. Please check your credentials."
    
    try:
        # Call the Sheets API
        result = sheets_service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id,
            range=range_name
        ).execute()
        
        values = result.get('values', [])
        
        if not values:
            return "No data found in the specified range."
        
        # Convert to a more readable format
        data_summary = f"Retrieved {len(values)} rows of data:\n\n"
        
        # Add headers if available
        if values:
            headers = values[0]
            data_summary += f"Columns: {', '.join(headers)}\n\n"
            
            # Add sample data (first few rows)
            sample_rows = min(5, len(values))
            data_summary += "Sample data:\n"
            for i, row in enumerate(values[:sample_rows]):
                # Ensure all rows have the same number of columns
                padded_row = row + [''] * (len(headers) - len(row))
                row_data = ' | '.join(padded_row[:len(headers)])
                data_summary += f"Row {i+1}: {row_data}\n"
            
            if len(values) > sample_rows:
                data_summary += f"... and {len(values) - sample_rows} more rows"
        
        return data_summary
        
    except HttpError as error:
        return f"Error reading sheet: {error}"
    except Exception as e:
        return f"Unexpected error: {e}"

@function_tool
def write_sheet_data(spreadsheet_id: str, range_name: str, values: List[List[str]]) -> str:
    """
    Write data to a Google Sheet.
    
    Args:
        spreadsheet_id: The ID of the Google Spreadsheet
        range_name: The range to write to (e.g., 'Sheet1!A1:C3')
        values: 2D array of values to write
    
    Returns:
        Success or error message
    """
    if not sheets_service:
        return "Error: Google Sheets service not available. Please check your credentials."
    
    try:
        body = {
            'values': values
        }
        
        result = sheets_service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=range_name,
            valueInputOption='USER_ENTERED',  # Parse formulas and format data
            body=body
        ).execute()
        
        updated_cells = result.get('updatedCells', 0)
        return f"Successfully updated {updated_cells} cells in range {range_name}"
        
    except HttpError as error:
        return f"Error writing to sheet: {error}"
    except Exception as e:
        return f"Unexpected error: {e}"

@function_tool
def append_sheet_data(spreadsheet_id: str, range_name: str, values: List[List[str]]) -> str:
    """
    Append data to a Google Sheet.
    
    Args:
        spreadsheet_id: The ID of the Google Spreadsheet
        range_name: The range to append to (e.g., 'Sheet1!A:C')
        values: 2D array of values to append
    
    Returns:
        Success or error message
    """
    if not sheets_service:
        return "Error: Google Sheets service not available. Please check your credentials."
    
    try:
        body = {
            'values': values
        }
        
        result = sheets_service.spreadsheets().values().append(
            spreadsheetId=spreadsheet_id,
            range=range_name,
            valueInputOption='USER_ENTERED',
            body=body
        ).execute()
        
        updated_cells = result.get('updates', {}).get('updatedCells', 0)
        return f"Successfully appended {len(values)} rows ({updated_cells} cells) to {range_name}"
        
    except HttpError as error:
        return f"Error appending to sheet: {error}"
    except Exception as e:
        return f"Unexpected error: {e}"

@function_tool
def clear_sheet_range(spreadsheet_id: str, range_name: str) -> str:
    """
    Clear data from a specific range in a Google Sheet.
    
    Args:
        spreadsheet_id: The ID of the Google Spreadsheet
        range_name: The range to clear (e.g., 'Sheet1!A1:C10')
    
    Returns:
        Success or error message
    """
    if not sheets_service:
        return "Error: Google Sheets service not available. Please check your credentials."
    
    try:
        result = sheets_service.spreadsheets().values().clear(
            spreadsheetId=spreadsheet_id,
            range=range_name
        ).execute()
        
        return f"Successfully cleared range {range_name}"
        
    except HttpError as error:
        return f"Error clearing sheet range: {error}"
    except Exception as e:
        return f"Unexpected error: {e}"

@function_tool
def analyze_sheet_data(spreadsheet_id: str, range_name: str = "A:Z") -> str:
    """
    Analyze data in a Google Sheet and provide insights.
    
    Args:
        spreadsheet_id: The ID of the Google Spreadsheet
        range_name: The range to analyze (default: all data)
    
    Returns:
        Data analysis summary
    """
    if not sheets_service:
        return "Error: Google Sheets service not available. Please check your credentials."
    
    try:
        # Get the data first
        result = sheets_service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id,
            range=range_name
        ).execute()
        
        values = result.get('values', [])
        
        if not values:
            return "No data found to analyze."
        
        # Basic analysis
        analysis = f"📊 **Sheet Data Analysis**\n\n"
        analysis += f"• Total rows: {len(values)}\n"
        
        if values:
            headers = values[0]
            analysis += f"• Total columns: {len(headers)}\n"
            analysis += f"• Column names: {', '.join(headers)}\n\n"
            
            # Analyze each column
            if len(values) > 1:  # Has data beyond headers
                data_rows = values[1:]
                analysis += "**Column Analysis:**\n"
                
                for col_idx, header in enumerate(headers):
                    col_values = []
                    for row in data_rows:
                        if col_idx < len(row) and row[col_idx].strip():
                            col_values.append(row[col_idx].strip())
                    
                    if col_values:
                        analysis += f"\n• **{header}**: {len(col_values)} non-empty values"
                        
                        # Try to detect if numeric
                        numeric_values = []
                        for val in col_values:
                            try:
                                numeric_values.append(float(val.replace(',', '')))
                            except ValueError:
                                break
                        
                        if len(numeric_values) == len(col_values) and numeric_values:
                            # Numeric column
                            avg_val = sum(numeric_values) / len(numeric_values)
                            analysis += f" (Numeric - Avg: {avg_val:.2f}, Min: {min(numeric_values)}, Max: {max(numeric_values)})"
                        else:
                            # Text column - show unique values count
                            unique_values = len(set(col_values))
                            analysis += f" (Text - {unique_values} unique values)"
        
        return analysis
        
    except HttpError as error:
        return f"Error analyzing sheet: {error}"
    except Exception as e:
        return f"Unexpected error during analysis: {e}"

@function_tool
def create_summary_report(spreadsheet_id: str, source_range: str, summary_range: str) -> str:
    """
    Create a summary report from data and write it to another range.
    
    Args:
        spreadsheet_id: The ID of the Google Spreadsheet
        source_range: Range to analyze (e.g., 'Data!A:Z')
        summary_range: Range to write summary (e.g., 'Summary!A1')
    
    Returns:
        Success message with summary details
    """
    if not sheets_service:
        return "Error: Google Sheets service not available. Please check your credentials."
    
    try:
        # Get source data
        result = sheets_service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id,
            range=source_range
        ).execute()
        
        values = result.get('values', [])
        
        if not values:
            return "No data found to summarize."
        
        # Create summary data
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        summary_data = [
            ["📊 Data Summary Report", ""],
            ["Generated:", timestamp],
            ["", ""],
            ["Total Rows:", str(len(values))],
            ["Total Columns:", str(len(values[0]) if values else 0)],
            ["", ""],
            ["Source Range:", source_range],
        ]
        
        if values and len(values) > 1:
            headers = values[0]
            summary_data.append(["Columns:", ", ".join(headers)])
        
        # Write summary
        body = {'values': summary_data}
        sheets_service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=summary_range,
            valueInputOption='USER_ENTERED',
            body=body
        ).execute()
        
        return f"Successfully created summary report in {summary_range} with {len(summary_data)} rows of summary data"
        
    except HttpError as error:
        return f"Error creating summary: {error}"
    except Exception as e:
        return f"Unexpected error: {e}"

print("✅ All Google Sheets tools created successfully!")

## 4. Building the Spreadsheet Agent

Now let's create our intelligent spreadsheet agent with all the Google Sheets tools.

In [ ]:
# Create the spreadsheet agent
spreadsheet_agent = Agent(
    name="Google Sheets Assistant",
    instructions="""
    You are an expert Google Sheets assistant specializing in data analysis, manipulation, and reporting.
    
    Your capabilities:
    1. Read data from Google Sheets and understand structure
    2. Write and append data to spreadsheets
    3. Analyze data and provide insights
    4. Create summary reports and visualizations
    5. Clear and manage spreadsheet ranges
    6. Help with data organization and cleanup
    
    Best practices:
    - Always read data first before making changes
    - Provide clear explanations of what you're doing
    - Suggest data improvements and organization tips
    - When analyzing data, be thorough and highlight key insights
    - Ask for confirmation before making destructive changes (like clearing data)
    
    When working with spreadsheets:
    - Use specific range notation (e.g., 'Sheet1!A1:C10')
    - Always validate spreadsheet IDs before operations
    - Provide helpful context about the data structure
    - Suggest next steps for data analysis or manipulation
    """,
    model="gpt-4o-mini",  # Use gpt-4o for more complex analysis
    tools=[
        read_sheet_data,
        write_sheet_data,
        append_sheet_data,
        clear_sheet_range,
        analyze_sheet_data,
        create_summary_report
    ],
    model_settings=ModelSettings(
        temperature=0.3,  # Lower temperature for more consistent data operations
        max_tokens=2500   # Allow for comprehensive responses
    )
)

print(f"✅ Spreadsheet Agent '{spreadsheet_agent.name}' created with {len(spreadsheet_agent.tools)} tools")
print("\nAgent capabilities:")
for i, tool in enumerate(spreadsheet_agent.tools, 1):
    print(f"  {i}. {tool.name}")

## 5. Basic Spreadsheet Operations

Let's test our spreadsheet agent with basic operations. **Note**: Replace `YOUR_SPREADSHEET_ID` with your actual Google Sheets ID.

In [ ]:
# Example spreadsheet ID - replace with your own
# You can find this in your Google Sheets URL: 
# https://docs.google.com/spreadsheets/d/YOUR_SPREADSHEET_ID/edit
EXAMPLE_SPREADSHEET_ID = "YOUR_SPREADSHEET_ID"  # Replace this!

# Example 1: Read existing data
print("=" * 60)
print("📖 Example 1: Reading Spreadsheet Data")
print("=" * 60)

if EXAMPLE_SPREADSHEET_ID != "YOUR_SPREADSHEET_ID":
    read_config = RunConfig(
        workflow_name="Spreadsheet Read Operation",
        trace_include_sensitive_data=False
    )
    
    result = Runner.run_sync(
        spreadsheet_agent,
        f"Please read the data from spreadsheet {EXAMPLE_SPREADSHEET_ID} and tell me what you find. Analyze the structure and content.",
        run_config=read_config
    )
    
    print(result.final_output)
else:
    print("⚠️ Please replace EXAMPLE_SPREADSHEET_ID with your actual Google Sheets ID")
    print("You can find this in your Google Sheets URL after '/d/' and before '/edit'")

In [ ]:
# Example 2: Create sample data
print("\n" + "=" * 60)
print("✍️ Example 2: Creating Sample Data")
print("=" * 60)

if EXAMPLE_SPREADSHEET_ID != "YOUR_SPREADSHEET_ID":
    create_data_query = f"""
    Please create a sample dataset in spreadsheet {EXAMPLE_SPREADSHEET_ID} with the following:
    
    1. Clear any existing data in range A1:E10
    2. Create a header row with: Name, Age, Department, Salary, Join_Date
    3. Add 5 rows of sample employee data
    4. Then analyze the data you just created
    
    Use realistic sample data and make sure to format it properly.
    """
    
    create_config = RunConfig(
        workflow_name="Create Sample Data",
        trace_include_sensitive_data=False
    )
    
    result = Runner.run_sync(
        spreadsheet_agent,
        create_data_query,
        run_config=create_config
    )
    
    print(result.final_output)
else:
    print("⚠️ Please set up your spreadsheet ID first")

## 6. Advanced Data Analysis with Tracing

Let's perform more complex data analysis operations with proper tracing.

In [ ]:
# Advanced analysis with tracing
async def comprehensive_data_analysis(spreadsheet_id: str):
    """Perform comprehensive data analysis with tracing"""
    
    with trace("Comprehensive Spreadsheet Analysis") as analysis_trace:
        print(f"Started analysis trace: {analysis_trace.id}")
        
        # Step 1: Initial data exploration
        print("\n📊 Step 1: Data Exploration...")
        exploration_result = await Runner.run(
            spreadsheet_agent,
            f"Analyze all data in spreadsheet {spreadsheet_id}. Provide detailed insights about the data structure, types, and patterns you observe.",
            run_config=RunConfig(
                workflow_name="Data Exploration",
                trace_include_sensitive_data=False
            )
        )
        
        # Step 2: Create summary report
        print("\n📋 Step 2: Creating Summary Report...")
        summary_result = await Runner.run(
            spreadsheet_agent,
            f"Create a comprehensive summary report for the data in spreadsheet {spreadsheet_id}. Write the summary to a new range called 'Summary!A1' in the same spreadsheet.",
            run_config=RunConfig(
                workflow_name="Summary Creation",
                trace_include_sensitive_data=False
            )
        )
        
        # Step 3: Recommendations
        print("\n💡 Step 3: Data Recommendations...")
        recommendations_result = await Runner.run(
            spreadsheet_agent,
            f"Based on your analysis of spreadsheet {spreadsheet_id}, provide specific recommendations for data improvements, additional analysis, or potential insights that could be extracted.",
            run_config=RunConfig(
                workflow_name="Recommendations",
                trace_include_sensitive_data=False
            )
        )
        
        print("\n" + "=" * 80)
        print("📊 COMPREHENSIVE ANALYSIS RESULTS")
        print("=" * 80)
        
        print("\n🔍 DATA EXPLORATION:")
        print("-" * 40)
        print(exploration_result.final_output)
        
        print("\n📋 SUMMARY REPORT:")
        print("-" * 40)
        print(summary_result.final_output)
        
        print("\n💡 RECOMMENDATIONS:")
        print("-" * 40)
        print(recommendations_result.final_output)
        
        return {
            "exploration": exploration_result.final_output,
            "summary": summary_result.final_output,
            "recommendations": recommendations_result.final_output
        }

# Run comprehensive analysis (uncomment when ready)
print("Comprehensive analysis function prepared.")
print("To run: await comprehensive_data_analysis(EXAMPLE_SPREADSHEET_ID)")

# Uncomment to run:
# if EXAMPLE_SPREADSHEET_ID != "YOUR_SPREADSHEET_ID":
#     import asyncio
#     analysis_results = await comprehensive_data_analysis(EXAMPLE_SPREADSHEET_ID)

## 7. Interactive Spreadsheet Session

Let's create an interactive session for working with spreadsheets.

In [ ]:
class SpreadsheetSession:
    """Interactive session for spreadsheet operations"""
    
    def __init__(self, spreadsheet_id: str, agent: Agent):
        self.spreadsheet_id = spreadsheet_id
        self.agent = agent
        self.session_history = []
        self.session_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    def execute_command(self, command: str) -> str:
        """Execute a spreadsheet command"""
        enhanced_command = f"""
        Working with spreadsheet: {self.spreadsheet_id}
        
        Previous operations in this session:
        {self._get_session_context()}
        
        Current request: {command}
        
        Please execute this request and provide clear feedback about what was accomplished.
        """
        
        config = RunConfig(
            workflow_name=f"Spreadsheet Session {self.session_id}",
            trace_include_sensitive_data=False,
            group_id=f"session_{self.session_id}"
        )
        
        result = Runner.run_sync(self.agent, enhanced_command, run_config=config)
        
        # Store in history
        self.session_history.append({
            "timestamp": datetime.now().isoformat(),
            "command": command,
            "result": result.final_output
        })
        
        return result.final_output
    
    def _get_session_context(self) -> str:
        """Get summary of recent session history"""
        if not self.session_history:
            return "No previous operations in this session."
        
        context = "Recent operations:\n"
        for i, item in enumerate(self.session_history[-3:], 1):  # Last 3 operations
            context += f"{i}. {item['command'][:100]}...\n"
        return context
    
    def get_session_summary(self) -> Dict[str, Any]:
        """Get complete session summary"""
        return {
            "session_id": self.session_id,
            "spreadsheet_id": self.spreadsheet_id,
            "total_operations": len(self.session_history),
            "history": self.session_history
        }

# Demo spreadsheet session
if EXAMPLE_SPREADSHEET_ID != "YOUR_SPREADSHEET_ID":
    print("=" * 60)
    print("🔄 Interactive Spreadsheet Session Demo")
    print("=" * 60)
    
    # Create session
    session = SpreadsheetSession(EXAMPLE_SPREADSHEET_ID, spreadsheet_agent)
    
    # Demo commands
    demo_commands = [
        "Read the current data and tell me what you see",
        "Add a new row with employee data: John Smith, 28, Engineering, 75000, 2024-01-15",
        "Calculate the average salary from all the data"
    ]
    
    for i, command in enumerate(demo_commands, 1):
        print(f"\n💬 Command {i}: {command}")
        print("🤖 Response:")
        response = session.execute_command(command)
        print(response[:300] + "..." if len(response) > 300 else response)
        print("-" * 40)
    
    print(f"\n📊 Session completed with {len(session.session_history)} operations")
else:
    print("⚠️ Set up your spreadsheet ID to run the interactive session demo")

print("\n✅ Interactive spreadsheet session functionality ready!")

## 8. Practical Use Cases

Let's explore some practical scenarios for using our spreadsheet agent.

In [ ]:
# Practical use case examples

def sales_data_processor(spreadsheet_id: str) -> str:
    """Process sales data with the spreadsheet agent"""
    query = f"""
    I have sales data in spreadsheet {spreadsheet_id}. Please:
    
    1. Read the sales data and identify the structure
    2. Calculate total sales, average order value, and top products
    3. Create a summary report in a new sheet/range
    4. Suggest any data quality improvements
    
    Provide detailed analysis and insights.
    """
    
    config = RunConfig(
        workflow_name="Sales Data Analysis",
        trace_include_sensitive_data=False
    )
    
    result = Runner.run_sync(spreadsheet_agent, query, run_config=config)
    return result.final_output

def inventory_manager(spreadsheet_id: str) -> str:
    """Manage inventory data"""
    query = f"""
    Help me manage inventory in spreadsheet {spreadsheet_id}:
    
    1. Check current inventory levels
    2. Identify items that are low in stock (less than 10 units)
    3. Calculate total inventory value
    4. Create a reorder report highlighting items that need restocking
    
    Format the results clearly and provide actionable recommendations.
    """
    
    config = RunConfig(
        workflow_name="Inventory Management",
        trace_include_sensitive_data=False
    )
    
    result = Runner.run_sync(spreadsheet_agent, query, run_config=config)
    return result.final_output

def budget_analyzer(spreadsheet_id: str) -> str:
    """Analyze budget data"""
    query = f"""
    Analyze the budget data in spreadsheet {spreadsheet_id}:
    
    1. Read the budget vs actual spending data
    2. Calculate variances (over/under budget)
    3. Identify the biggest spending categories
    4. Create a variance report with recommendations
    5. Suggest budget optimizations
    
    Provide financial insights and actionable recommendations.
    """
    
    config = RunConfig(
        workflow_name="Budget Analysis",
        trace_include_sensitive_data=False
    )
    
    result = Runner.run_sync(spreadsheet_agent, query, run_config=config)
    return result.final_output

# Demo the use cases
print("💼 Practical Use Cases for Spreadsheet Agent")
print("=" * 50)

use_cases = {
    "Sales Data Processing": sales_data_processor,
    "Inventory Management": inventory_manager,
    "Budget Analysis": budget_analyzer
}

for name, func in use_cases.items():
    print(f"\n📊 {name}:")
    print(f"   Function: {func.__name__}()")
    print(f"   Purpose: {func.__doc__}")

print("\n✅ All use case functions are ready to use with your spreadsheet ID!")

## 9. Best Practices and Tips

Here are important best practices when working with the Google Sheets Agent.

In [ ]:
# Best practices for spreadsheet agents

# 1. Error handling for Google Sheets operations
@function_tool
def safe_sheet_operation(spreadsheet_id: str, operation: str) -> str:
    """
    Example of safe sheet operation with comprehensive error handling.
    
    Args:
        spreadsheet_id: The spreadsheet ID
        operation: Description of the operation
    
    Returns:
        Result or detailed error message
    """
    try:
        # Validate spreadsheet ID format
        if not spreadsheet_id or len(spreadsheet_id) < 20:
            return "Error: Invalid spreadsheet ID format"
        
        # Your operation logic here
        return f"Successfully completed: {operation}"
        
    except HttpError as e:
        if e.resp.status == 404:
            return "Error: Spreadsheet not found. Please check the ID and ensure it's shared with the service account."
        elif e.resp.status == 403:
            return "Error: Permission denied. Please share the spreadsheet with the service account email."
        else:
            return f"Google Sheets API Error: {e}"
    except Exception as e:
        return f"Unexpected error: {e}"

# 2. Configuration management
class SheetsConfig:
    """Configuration management for Google Sheets operations"""
    
    def __init__(self, credentials_path: str = "credentials.json"):
        self.credentials_path = credentials_path
        self.scopes = ['https://www.googleapis.com/auth/spreadsheets']
        self.service = None
    
    def get_service(self):
        """Get or create Google Sheets service"""
        if not self.service:
            self.service = setup_sheets_service()
        return self.service
    
    def validate_spreadsheet_id(self, spreadsheet_id: str) -> bool:
        """Validate spreadsheet ID format"""
        return len(spreadsheet_id) >= 20 and "/" not in spreadsheet_id

# 3. Data validation helpers
def validate_range_format(range_name: str) -> bool:
    """Validate Google Sheets range format"""
    # Basic validation for range format
    valid_patterns = [
        r'^[A-Z]+:[A-Z]+$',  # A:Z
        r'^[A-Z]+\d+:[A-Z]+\d+$',  # A1:C10
        r'^\w+![A-Z]+\d+:[A-Z]+\d+$',  # Sheet1!A1:C10
    ]
    import re
    return any(re.match(pattern, range_name) for pattern in valid_patterns)

def format_sheet_data(values: List[List[str]]) -> str:
    """Format sheet data for better readability"""
    if not values:
        return "No data found"
    
    formatted = "\n".join([
        " | ".join(row) for row in values[:10]  # First 10 rows
    ])
    
    if len(values) > 10:
        formatted += f"\n... and {len(values) - 10} more rows"
    
    return formatted

# 4. Session management best practices
class SpreadsheetSessionManager:
    """Advanced session management for spreadsheet operations"""
    
    def __init__(self):
        self.active_sessions = {}
        self.config = SheetsConfig()
    
    def create_session(self, spreadsheet_id: str, user_id: str = "default") -> str:
        """Create a new spreadsheet session"""
        if not self.config.validate_spreadsheet_id(spreadsheet_id):
            raise ValueError("Invalid spreadsheet ID")
        
        session_id = f"{user_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        self.active_sessions[session_id] = {
            "spreadsheet_id": spreadsheet_id,
            "user_id": user_id,
            "created_at": datetime.now(),
            "operations": []
        }
        return session_id
    
    def log_operation(self, session_id: str, operation: str, result: str):
        """Log an operation in the session"""
        if session_id in self.active_sessions:
            self.active_sessions[session_id]["operations"].append({
                "timestamp": datetime.now(),
                "operation": operation,
                "result": result
            })

# Example usage of best practices
print("📋 Google Sheets Agent Best Practices:")
print("=" * 45)

best_practices = [
    "✅ Always validate spreadsheet IDs before operations",
    "✅ Handle Google Sheets API errors gracefully",
    "✅ Use specific range notation for clarity",
    "✅ Implement proper error handling and user feedback",
    "✅ Validate data formats before writing to sheets",
    "✅ Use tracing for complex multi-step operations",
    "✅ Keep session history for better context",
    "✅ Provide clear success/error messages",
    "✅ Ask for confirmation before destructive operations",
    "✅ Share spreadsheets with service account email"
]

for practice in best_practices:
    print(f"  {practice}")

print("\n💡 Pro Tips:")
print("  • Test with a sample spreadsheet first")
print("  • Keep credentials.json secure and never commit to git")
print("  • Use environment variables for sensitive data")
print("  • Monitor API quota usage for large operations")
print("  • Implement rate limiting for bulk operations")

print("\n✅ Best practices guide completed!")

## 10. Summary and Next Steps

### What We've Accomplished

In this tutorial, we've built a comprehensive Google Sheets agent using the OpenAI Agents SDK:

1. **Google Sheets Integration** - Set up service account authentication and API access
2. **Custom Tools** - Created 6 powerful tools for sheet operations using `@function_tool`
3. **Intelligent Agent** - Built an agent specialized in spreadsheet data analysis and manipulation
4. **Practical Examples** - Demonstrated real-world use cases and workflows
5. **Best Practices** - Implemented proper error handling, tracing, and session management

### Key Features

Our spreadsheet agent can:
- **Read & Analyze** - Parse spreadsheet data and provide insights
- **Write & Update** - Add new data and modify existing content
- **Create Reports** - Generate summary reports and analysis
- **Data Validation** - Ensure data quality and format consistency
- **Session Management** - Maintain context across operations

### Next Steps

To extend this spreadsheet agent further:

1. **Advanced Analytics**
   - Add statistical analysis tools
   - Implement data visualization
   - Create predictive models

2. **Integration Expansion**
   - Connect with other Google Workspace apps
   - Add database connectors
   - Integrate with external APIs

3. **Automation**
   - Schedule regular data updates
   - Set up data validation rules
   - Create automated reports

4. **Security & Scale**
   - Implement user authentication
   - Add audit logging
   - Handle rate limiting

### Resources

- [Google Sheets API Documentation](https://developers.google.com/sheets/api)
- [OpenAI Agents SDK Documentation](https://openai.github.io/openai-agents-python/)
- [Google Cloud Console](https://console.cloud.google.com/)

### Quick Start Function

Here's a simple function to get started quickly:

In [ ]:
def quick_sheet_analysis(spreadsheet_id: str, task: str = "analyze") -> str:
    """
    Quick spreadsheet analysis function.
    
    Args:
        spreadsheet_id: Your Google Sheets ID
        task: Type of analysis ('analyze', 'summarize', 'clean')
    
    Returns:
        Analysis results
    """
    task_instructions = {
        "analyze": "Analyze the data structure, identify patterns, and provide key insights",
        "summarize": "Create a comprehensive summary report of the data",
        "clean": "Identify data quality issues and suggest improvements"
    }
    
    query = f"""
    Working with spreadsheet: {spreadsheet_id}
    
    Task: {task_instructions.get(task, task_instructions['analyze'])}
    
    Please read the data and provide detailed feedback with actionable insights.
    """
    
    config = RunConfig(
        workflow_name=f"Quick Sheet {task.title()}",
        trace_include_sensitive_data=False
    )
    
    result = Runner.run_sync(spreadsheet_agent, query, run_config=config)
    return result.final_output

# Example usage demonstration
print("🚀 Quick Start Example:")
print("=" * 50)
print("# Replace with your actual spreadsheet ID")
print('spreadsheet_id = "1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms"')
print('')
print('# Quick analysis')
print('result = quick_sheet_analysis(spreadsheet_id, "analyze")')
print('print(result)')

print("\n" + "=" * 70)
print("🎉 Congratulations! You've built a powerful Google Sheets Agent!")
print("=" * 70)

if EXAMPLE_SPREADSHEET_ID != "YOUR_SPREADSHEET_ID":
    print(f"\n🔗 Your agent is ready to work with spreadsheet: {EXAMPLE_SPREADSHEET_ID}")
else:
    print("\n⚠️ Remember to set up your spreadsheet ID and credentials to start using the agent!")